In [ ]:
#rows represent individual records (ex. student info)
#columns represent attributes (name, gender, ID..)

#primary key is a unique key for for ex) student ID
#foreign key: to link tables together. so if u have students table amd courses table, u can use
# foreign key in courses to refer to a student in student table


# Database refresher

#### Relationships between data
* Let's create a database to store the information about students and the courses that they enroll in.

- The `student` table that stores student information.
- The `course` table that stores course information.
- The `enrollment` table that stores the relationship between students and courses.

The following Entity Relationship Diagram (ERD) illustrates tables:student, course, and enrollment.
You can create it in Lucidchart (SJSU has license for students at SJSU). You can find it in [one.sjsu.edu](https://one.sjsu.edu)

<img src="ImgER.png" width=450>
<br>
<img src="ERD_Cardinality.png" width=300>



#### [Data types in SQLite](https://www.sqlite.org/datatype3.html)
There are 5 data types for values stored in an SQLite database:

- NULL. The value is a NULL value.

- INTEGER. The value is a signed integer, stored in 1, 2, 3, 4, 6, or 8 bytes depending on the magnitude of the value.

- REAL. The value is a floating point value, stored as an 8-byte IEEE floating point number.

- TEXT. The value is a text string, stored using the database encoding (UTF-8, UTF-16BE or UTF-16LE).

- BLOB. The value is a blob of data, stored exactly as it was input.

In [1]:
import sqlite3
from pathlib import Path

Path('newdb.db').touch()
conn = sqlite3.connect('newdb.db')
conn.execute("PRAGMA foreign_keys = ON")
c = conn.cursor()

More information about [SQLite Foreign Key support](https://www.sqlite.org/foreignkeys.html).


Operations on database


In [13]:
SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS student (
  student_id INTEGER,
  firstname TEXT,
  lastname TEXT,
  email TEXT,
  PRIMARY KEY (student_id))'''
c.execute(SQL_CreateTable)

# NOTE: the additional IF NOT EXISTS after CREATE TABLE


In [14]:
# Add data
SQL_InsertStmt = """INSERT INTO student VALUES
                 (1, 'Tom', 'Riddle', 'tom@gmail.com'),
                 (8, 'Harry', 'Potter', 'harry@gmail.com')"""

c.execute(SQL_InsertStmt)

In [15]:
#specifiying fields other than Primary Key.
SQL_InsertStmt = """INSERT INTO student (firstname, lastname, email) VALUES
                 ( 'Hermoine', 'Granger', 'hermoine@sjsumail.com')"""
c.execute(SQL_InsertStmt)

In [16]:
c.execute("SELECT * FROM student")
results = c.fetchall()
print(results) #adds hermoine with corresponding next value for id

[(1, 'Tom', 'Riddle', 'tom@gmail.com'), (8, 'Harry', 'Potter', 'harry@gmail.com'), (9, 'Hermoine', 'Granger', 'hermoine@sjsumail.com')]


Let's create the other two tables: course and enrollment.

In [17]:
SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS course (
             course_id INTEGER PRIMARY KEY,
             name TEXT NOT NULL,
             desc TEXT NOT NULL
             )'''

c.execute(SQL_CreateTable)

In [18]:
SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS enrollment (
             student_id INTEGER,
             course_id INTEGER,
             term TEXT NOT NULL,
             year INTEGER,
             FOREIGN KEY (student_id)
                 REFERENCES student (student_id)
                     ON DELETE CASCADE
                     ON UPDATE NO ACTION,
             FOREIGN KEY (course_id)
                 REFERENCES course (course_id)
                     ON DELETE CASCADE
                     ON UPDATE NO ACTION
             )'''
c.execute(SQL_CreateTable)

In [19]:
conn.commit()

`DELETE CASCADE`: When we create a foreign key using this option, it deletes the referencing rows in the child table when the referenced row is deleted in the parent table which has a primary key.

`UPDATE NO ACTION`: When we create a foreign key using this option, the foreign key constraint (NO ACTION) behaves whenever the parent key is updated.

#### Activity : Try out a couple of insert statements into the course and enrollment tables. Test the behaviour of foreign key by trying to insert enrollment for non-existent student_id.


 #### Working with csv files and sqlite

In [2]:
import pandas as pd
course=pd.read_csv('course.csv')
course

,course_id,name,desc
0,12345,CS122,Advanced Python Prog.
1,54321,CS156,Intro. to AI.


In [3]:
# insert the data from dataframe to database table `course`
course.to_sql('course', conn, if_exists='append', index=False)

2

In [6]:
c.execute("SELECT * FROM course")

results = c.fetchall()
print(results)

[(12345, 'CS122', 'Advanced Python Prog.'), (54321, 'CS156', 'Intro. to AI.')]


Similarly we can have enrollment relations in a csv and insert into database directly.


In [8]:
enroll=pd.read_csv('enroll.csv')

# insert the data from dataframe to database table `enrollment`
enroll.to_sql('enrollment', conn, if_exists='append', index=False)

2

In [9]:
c.execute("SELECT * FROM enrollment")
results = c.fetchall()
print(results)



[(8, 12345, 'Summer', 2025), (1, 54321, 'Spring', 2025)]


In [10]:

SQL_InsertStmt = """INSERT INTO enrollment VALUES
                 ( 200,12345,'Hermoine',2025)"""
c.execute(SQL_InsertStmt)

c.execute("SELECT * FROM enrollment")
results = c.fetchall()
print(results)

In [11]:
c.execute("SELECT * FROM enrollment")
results = c.fetchall()
print(results)

[(8, 12345, 'Summer', 2025), (1, 54321, 'Spring', 2025), (200, 12345, 'Hermoine', 2025)]


In [20]:
c.execute("SELECT * FROM student")
results = c.fetchall()
print(results)

[(1, 'Tom', 'Riddle', 'tom@gmail.com'), (8, 'Harry', 'Potter', 'harry@gmail.com'), (9, 'Hermoine', 'Granger', 'hermoine@sjsumail.com')]


#### Joins in SQL, simple example
There are many joins in SQL, you must experiment with them and brush up the basics outside of class.


In [21]:
SQL_JointQuery = """SELECT student.firstname, student.lastname,
                course.name FROM student
                JOIN enrollment using (student_id)
                JOIN course using (course_id)
                WHERE student.lastname='Potter'
                """
c.execute(SQL_JointQuery)
joint_results = c.fetchall()
print(joint_results)

[('Harry', 'Potter', 'CS122')]


Do not forget to save changes to database with the following:

In [22]:
conn.commit() # commit the current transaction
conn.close() # close the connection

# Object Relational Mapping (ORM)


- ORM stands for **O**bject **R**elational **M**apping - is a technique for querying and manipulating data from a database using an object-oriented paradigm. ORM is usually referred as a library that implements the Object-Relational Mapping technique, hence the phrase "an ORM".

### The benefits of using an ORM
- It fits in your natural way of coding and avoids writing poorly-formed SQL.
- It abstracts the DB system, so you can change it whenever you want.
- It forces one to write <a href="https://en.wikipedia.org/wiki/Model%E2%80%93view%E2%80%93controller">MVC</a> code, which, in the end, makes your code a little cleaner.
- The model is weakly bound to the rest of the application, so you can change it or use it anywhere else.
- It lets you use OOP goodness like data inheritance.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/MVC-Process.svg/300px-MVC-Process.svg.png" width=250>

## SQLAlchemy
[SQLAlchemy](https://docs.sqlalchemy.org/en/14/) automates redundant database tasks and provide Python object-based interfaces to the data while still allows the developer control of the database and access to the underlying SQL. In this lecture, we will look at some basic examples of storing data into a relational database and then retrieving it with SQLAlchemy.


In [23]:
# !pip install sqlalchemy

In [24]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

# Creating a database (SQLite for simplicity)
engine = create_engine("sqlite:///Univ.db")

# Base class for ORM models
Base = declarative_base()

Defining an ORM Model (Table Representation)


In [25]:
from sqlalchemy import Column, Integer, String

class Student(Base):
    __tablename__ = "students"

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    age = Column(Integer)

# Creating the tables
Base.metadata.create_all(engine)


Creating a Session (Database Interaction)


In [26]:
Session = sessionmaker(bind=engine)
session = Session()


Inserting, Querying, filtering, deleting, Update and other operations have similar syntax. For example:


In [27]:
new_student = Student(name="Alice", age=22)
session.add(new_student)
session.commit()


In [28]:
students = session.query(Student).all()
for student in students:
    print(student.id, student.name, student.age)

1 Alice 22


In [29]:
student = session.query(Student).filter_by(name="Alice").first()
print(student.id, student.name, student.age)


1 Alice 22


In [30]:
#update
student = session.query(Student).filter_by(name="Alice").first()
student.age = 23
session.commit()


In [31]:
#delete
student = session.query(Student).filter_by(name="Alice").first()
session.delete(student)
session.commit()


Foreign Key and Relationships

* One-to-Many Relationship (Students and Courses)

In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Course(Base):
    __tablename__ = "courses"

    id = Column(Integer, primary_key=True)
    title = Column(String, nullable=False)
    student_id = Column(Integer, ForeignKey("students.id"))

    student = relationship("Student", back_populates="courses")

# Add a relationship in Student
Student.courses = relationship("Course", back_populates="student", cascade="all, delete")

# Create tables
Base.metadata.create_all(engine)


/var/folders/0l/tb4bmv7s03z64wdw6837m9rw0000gs/T/ipykernel_3307/3523872684.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Course, and will be replaced in the string-lookup table.
  class Course(Base):


InvalidRequestError: Table 'courses' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

* Many to many

In [ ]:
from sqlalchemy import Table

# Association table (linking students and subjects)
student_subject = Table(
    "student_subject", Base.metadata,
    Column("student_id", Integer, ForeignKey("students.id")),
    Column("subject_id", Integer, ForeignKey("subjects.id"))
)

class Subject(Base):
    __tablename__ = "subjects"

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)

    students = relationship("Student", secondary=student_subject, back_populates="subjects")

Student.subjects = relationship("Subject", secondary=student_subject, back_populates="students")

# Create tables
Base.metadata.create_all(engine)


Rollback option

In [ ]:
try:
    new_student = Student(name=None, age=25)  # Name cannot be NULL
    session.add(new_student)
    session.commit()
except Exception as e:
    session.rollback()  # Undo the transaction
    print("Error:", e)


Error: (sqlite3.IntegrityError) NOT NULL constraint failed: students.name
[SQL: INSERT INTO students (name, age) VALUES (?, ?)]
[parameters: (None, 25)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
